In [ ]:
train_dir = '/teamspace/studios/this_studio/dataset/supervised/dataset/train'
test_dir = '/teamspace/studios/this_studio/dataset/supervised/dataset/test'

In [ ]:
import shutil
import os

def extract_zip(zip_path, extract_to):
    """
    Extracts the ZIP file to the specified folder.
    
    Args:
        zip_path (str): Path to the ZIP file.
        extract_to (str): Folder where the ZIP file will be extracted.
    """
    # Check if the ZIP file exists
    if not os.path.exists(zip_path):
        raise FileNotFoundError(f"{zip_path} not found.")
    
    # Check if the extraction target folder exists, create it if not
    if not os.path.exists(extract_to):
        os.makedirs(extract_to)
    
    # Open the ZIP file and extract its contents
    shutil.unpack_archive(zip_path, extract_to)
    print(f"ZIP file extracted to {extract_to} folder.")

# Example usage
zip_file_path = '/teamspace/studios/this_studio/archive.zip'
extract_folder = '/teamspace/studios/this_studio/dataset'
extract_zip(zip_file_path, extract_folder)


In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.models import efficientnet_b0
from tqdm.auto import tqdm


In [ ]:

# Define transformations
transform_train = transforms.Compose([
    transforms.RandomRotation(degrees=(-45, 45)),  # Random rotation within -45 to 45 degrees
    transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.1),  # Random color changes
    transforms.Resize((224, 224)),  # Resize to 224x224 pixels
    transforms.ToTensor(),  # Convert the image to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the tensor
])

transform_val = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224 pixels
    transforms.ToTensor(),  # Convert the image to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the tensor
])

# Load datasets
train_dataset = ImageFolder(root=train_dir, transform=transform_train)
val_dataset = ImageFolder(root=test_dir, transform=transform_val)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Load a pre-trained EfficientNet-B0 model
model = efficientnet_b0(weights='IMAGENET1K_V1')

# Replace the final fully connected layer
num_features = model.classifier[1].in_features
num_classes = len(train_dataset.classes)  # Automatically determine number of classes
model.classifier = nn.Sequential(
    nn.Dropout(p=0.2),
    nn.Linear(num_features, num_classes)
)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Define the learning rate scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
# Training loop
num_epochs = 15

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in tqdm(train_loader):
        # Move data to GPU if available
        images, labels = images.to(device), labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        loss = criterion(outputs, labels)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    scheduler.step()  # Update learning rate
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}, train_acc: {100 * correct / total}%")
    
    if (epoch % 3 == 0):
        # Validation loop
        model.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            for images, labels in tqdm(val_loader):
                # Move data to GPU if available
                images, labels = images.to(device), labels.to(device)

                # Forward pass
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

            print(f'Validation Accuracy: {100 * correct / total}%')

# Save the trained model
torch.save(model.state_dict(), 'efficientnet_b0_custom_1.pth')


Epoch [1/10], Loss: 0.7787453474622246, train_acc: 77.18888888888888%
Validation Accuracy: 91.2%
Epoch [2/10], Loss: 0.33777626856157544, train_acc: 89.88888888888889%
Epoch [3/10], Loss: 0.26598419727929307, train_acc: 91.47777777777777%
Epoch [4/10], Loss: 0.22585981896688753, train_acc: 92.82222222222222%
Validation Accuracy: 93.7%
Epoch [5/10], Loss: 0.2021464193239808, train_acc: 93.86666666666666%
Epoch [6/10], Loss: 0.09893491388965689, train_acc: 96.8%
Epoch [7/10], Loss: 0.06436489645188595, train_acc: 97.96666666666667%
Validation Accuracy: 96.7%
Epoch [8/10], Loss: 0.054149052723482624, train_acc: 98.23333333333333%
Epoch [9/10], Loss: 0.04353517838687709, train_acc: 98.78888888888889%
Epoch [10/10], Loss: 0.0374356541990679, train_acc: 98.86666666666666%
Validation Accuracy: 96.8%

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.models import efficientnet_b0
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [ ]:
# Validation loop
import matplotlib.pyplot as plt
import seaborn as sns

model.eval()
all_labels = []
all_predictions = []
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in tqdm(val_loader):
        # Move data to GPU if available
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

    print(f'Validation Accuracy: {100 * correct / total}%')
    
    # Calculate additional metrics
    accuracy = accuracy_score(all_labels, all_predictions)
    precision = precision_score(all_labels, all_predictions, average='weighted')
    recall = recall_score(all_labels, all_predictions, average='weighted')
    f1 = f1_score(all_labels, all_predictions, average='weighted')
    cm = confusion_matrix(all_labels, all_predictions)
    
    print(f'Validation Metrics: Accuracy: {accuracy*100}%, Precision: {precision*100}%, Recall: {recall*100}%, F1 Score: {f1*100}%')
    
# Plot the confusion matrix
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=train_dataset.classes, yticklabels=train_dataset.classes)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()

Validation Accuracy: 96.8%
Validation Metrics: Accuracy: 96.8%, Precision: 96.87781133598462%, Recall: 96.8%, F1 Score: 96.79142002572321%

##You can see confusion matrix in output.png

In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image

image_path = '/teamspace/studios/this_studio/yalikavak-marina-6-350x250.jpg'
image = Image.open(image_path)

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224 pixels
    transforms.ToTensor(),  # Convert the image to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the tensor
])

# Apply the transformations
image = transform(image).unsqueeze(0)  # Add a batch dimension

# Move the image to the same device as the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
image = image.to(device)

# Forward pass
with torch.no_grad():
    outputs = model(image)
    _, predicted = torch.max(outputs, 1)

# Interpret the output
class_names = train_dataset.classes  # Assuming you have a list of class names
predicted_class = class_names[predicted.item()]
print(f'Predicted class: {predicted_class}')
print(f'Actual class: Port') ## port image

Predicted class: Port
Actual class: Port